In [4]:
import csv
data = {}
with open("/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv") as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[1].lower()
    if key in data:
      data[key].append(row[2])
    else:
      data[key] = [row[2]]

In [5]:
classes_list = list(data.keys())
classes_list

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [26]:
import os
import pandas as pd

folder = "/kaggle/input/state-farm-distracted-driver-detection/imgs/test"
filenames = []
labels = []

for filename in os.listdir(folder):
    filenames.append(os.path.join(folder, filename))
    labels.append('c0')

df = pd.DataFrame({
    "filename": filenames,
    "label": labels
})

print(df.head())

                                            filename label
0  /kaggle/input/state-farm-distracted-driver-det...    c0
1  /kaggle/input/state-farm-distracted-driver-det...    c0
2  /kaggle/input/state-farm-distracted-driver-det...    c0
3  /kaggle/input/state-farm-distracted-driver-det...    c0
4  /kaggle/input/state-farm-distracted-driver-det...    c0


In [34]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(10, activation='softmax')
])
model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 98, 98, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 49, 49, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 47, 47, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │     6,554,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,588,458 (25.13 MB)

 Trainable params: 6,588,458 (25.13 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
train_dir = '/kaggle/input/state-farm-distracted-driver-detection/imgs/train'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

Found 22424 images belonging to 10 classes.


In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

val_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

es = EarlyStopping(monitor='val_accuracy', patience=2, min_delta=0.01, mode="max",
    restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=10,
    verbose=1,
    validation_data=val_generator,
    callbacks=[es]
)

Found 22424 images belonging to 10 classes.
Epoch 1/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 105s 600ms/step - acc: 0.9389 - loss: 0.2125 - val_acc: 0.9729 - val_loss: 0.0897
Epoch 2/10


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: acc,loss,val_acc,val_loss
  current = self.get_monitor_value(logs)


176/176 ━━━━━━━━━━━━━━━━━━━━ 103s 586ms/step - acc: 0.9740 - loss: 0.0828 - val_acc: 0.9732 - val_loss: 0.0919
Epoch 3/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 105s 596ms/step - acc: 0.9855 - loss: 0.0506 - val_acc: 0.9895 - val_loss: 0.0333
Epoch 4/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 105s 600ms/step - acc: 0.9857 - loss: 0.0480 - val_acc: 0.9909 - val_loss: 0.0253
Epoch 5/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 104s 590ms/step - acc: 0.9884 - loss: 0.0362 - val_acc: 0.9871 - val_loss: 0.0370
Epoch 6/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 104s 592ms/step - acc: 0.9905 - loss: 0.0307 - val_acc: 0.9905 - val_loss: 0.0344
Epoch 7/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 105s 596ms/step - acc: 0.9884 - loss: 0.0362 - val_acc: 0.9926 - val_loss: 0.0212
Epoch 8/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 106s 602ms/step - acc: 0.9932 - loss: 0.0208 - val_acc: 0.9836 - val_loss: 0.0595
Epoch 9/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 105s 599ms/step - acc: 0.9870 - loss: 0.0461 - val_acc: 0.9965 - val_loss: 0.0133
Epoch 10/10
176/176 ━━━━━━━━━━━━━━━

In [ ]:
# Alternative: Manual loading without flow_from_directory
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

image_files = [f for f in os.listdir(folder) if f.endswith(('.jpg', '.png', '.jpeg'))]
predictions = []
for file in image_files:
    img_path = os.path.join(folder, file)
    img = load_img(img_path, target_size=(100,100))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Match training preprocessing
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    pred = model.predict(img_array)
    pred_class_idx = np.argmax(pred, axis=1)[0]
    pred_class = classes_list[pred_class_idx]
    predictions.append((file, pred_class))
    print(f'Image: {file}, Predicted Class: {pred_class}')

# Save predictions
with open('predictions.txt', 'w') as f:
    for file, pred in predictions:
        f.write(f'{file}: {pred}\n')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: img_96590.jpg, Predicted Class: c1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Image: img_32366.jpg, Predicted Class: c5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Image: img_99675.jpg, Predicted Class: c9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image: img_85937.jpg, Predicted Class: c9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Image: img_73903.jpg, Predicted Class: c7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image: img_89410.jpg, Predicted Class: c4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Image: img_70462.jpg, Predicted Class: c9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Image: img_45622.jpg, Predicted Class: c2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Image: img_55507.jpg, Predicted Class: c0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image: img_34204.jpg, Predicted Class: c8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image: img_16279.jpg, Predicted Class: c3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Image: img_73988.jpg, Predicted Class: c7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Im